# Import libraries

In [35]:
import pandas as pd
from skimage.feature import blob_log
from numpy import unravel_index
from math import sqrt
from geopandas import read_file
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
%matplotlib inline

# Read in world fishnet shapefile

In [36]:
fishnet_path = "../../../../00_Data/01_Processed/World_Fishnet-10x10deg_WGS84/WorldFishnet_10x10.shp"
fishnet = read_file(fishnet_path)
del fishnet['Id']

In [37]:
fishnet.head()

,geometry
0,"POLYGON ((-180 -59.9999999999918, -180 -49.999..."
1,"POLYGON ((-170 -59.9999999999918, -170 -49.999..."
2,"POLYGON ((-160 -59.9999999999918, -160 -49.999..."
3,"POLYGON ((-150 -59.9999999999918, -150 -49.999..."
4,"POLYGON ((-140 -59.9999999999918, -140 -49.999..."


# Define path to difference rasters

In [38]:
GPW_minus_BM_path = '../../../../00_Data/01_Processed/GPW_Minus_BM_gte_0_20180220/GPW_minus_BM_gte_0.tif'
BM_minus_GPW_path = '../../../../00_Data/01_Processed/BM_Minus_GPW_gte_0_20180220/BM_minus_GPW_gte_0.tif'

# Blob detection

In [ ]:
%%time

# store results in a list
results = []
errors = []

count = 0

# for every cell in grid
for i in fishnet.index:
    
    try:
        
        # print cell number
        print(i, "(", count, "/", len(fishnet), ")")
        count += 1

        # get cell polygon
        polygon = fishnet.loc[[i]]

        # get zonal stats for GPW minus BM
        GPW_minus_BM_stats = zonal_stats(
                            polygon,
                            GPW_minus_BM_path,
                            all_touched=True,
                            raster_out=True
                        )[0]

        # get zonal stats for BM minus GPW
        BM_minus_GPW_stats = zonal_stats(
                            polygon,
                            BM_minus_GPW_path,
                            all_touched=True,
                            raster_out=True
                        )[0]

        # function to convert row, col pixel coordinates to lat, lon
        def pixel2coord(transform, row, col):
            ''' converts row col to lat, lon'''
            coff, roff = (0.5, 0.5)
            x, y = transform * transform.translation(coff, roff) * (col, row)
            return y, x

        # function to detect blobs in image
        def detect_blobs(zonal_stats_object, threshold):
            array = zonal_stats_object['mini_raster_array']
            array.mask = False
            blobs = blob_log(array, max_sigma=30, num_sigma=10, threshold=threshold)
            # Compute blob radius in the 3rd column.
            blobs[:, 2] = blobs[:, 2] * sqrt(2)
            # Convert pixels to lat, lons
            affine_transform = zonal_stats_object['mini_raster_affine']
            blob_coords = [pixel2coord(affine_transform, i[0], i[1]) for i in blobs]
            return blob_coords
        
        # blob detection for people no lights
        people_no_lights = detect_blobs(GPW_minus_BM_stats, 
                                    threshold=0.3)
        print(len(people_no_lights), "people-no-lights blobs found")

        # build dataframe containing people no lights blobs
        people_no_lights_lats = [i[0] for i in people_no_lights]
        people_no_lights_lons = [i[1] for i in people_no_lights]
        people_no_lights_df = pd.DataFrame()
        people_no_lights_df['latitude'] = people_no_lights_lats
        people_no_lights_df['longitude'] = people_no_lights_lons
        people_no_lights_df['cell_id'] = i
        people_no_lights_df['category'] = "people_no_lights"
        people_no_lights_df['is_max'] = 0

        # blob detection for lights no people
        lights_no_people = detect_blobs(BM_minus_GPW_stats,
                                   threshold=0.1)
        print(len(lights_no_people), "lights-no-people blobs found")

        # build dataframe containing lights no people blobs
        lights_no_people_lats = [i[0] for i in lights_no_people]
        lights_no_people_lons = [i[1] for i in lights_no_people]
        lights_no_people_df = pd.DataFrame()
        lights_no_people_df['latitude'] = lights_no_people_lats
        lights_no_people_df['longitude'] = lights_no_people_lons
        lights_no_people_df['cell_id'] = i
        lights_no_people_df['category'] = "lights_no_people"
        lights_no_people_df['is_max'] = 0

        # concatenate the four dataframes into one dataframe
        this_df = pd.concat([people_no_lights_df, lights_no_people_df])

        # append the concatenated dataframe to the results list
        results.append(this_df)
        
    except StandardError as e:
        
        # if error append cell id to errors list
        print("Error:", e, i)
        errors.append(i)

0 ( 0 / 555 )


/Users/darebrawley/anaconda3/envs/geo/lib/python3.6/site-packages/rasterstats/main.py:142: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
/Users/darebrawley/anaconda3/envs/geo/lib/python3.6/site-packages/rasterstats/io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)
/Users/darebrawley/anaconda3/envs/geo/lib/python3.6/site-packages/rasterstats/io.py:294: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/darebrawley/anaconda3/envs/geo/lib/python3.6/site-packages/rasterstats/main.py:161: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(floa

0 people-no-lights blobs found
0 lights-no-people blobs found
1 ( 1 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
2 ( 2 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
3 ( 3 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
4 ( 4 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
5 ( 5 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
6 ( 6 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
7 ( 7 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
8 ( 8 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
9 ( 9 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
10 ( 10 / 555 )
0 people-no-lights blobs found
5 lights-no-people blobs found
11 ( 11 / 555 )
0 people-no-lights blobs found
5 lights-no-people blobs found
12 ( 12 / 555 )
0 people-no-lights blobs found
0 lights-no-people blobs found
13 ( 13 / 555 )
0 pe

In [ ]:
print len(results), "successes"
print len(errors), "errors"

In [ ]:
blobs = pd.concat(results)
blobs.reset_index(drop=True, inplace=True)
blobs.head()

In [ ]:
blobs.to_csv("../output/world_blobs.csv")

In [ ]:
blobs.shape